In [1]:
import torch
import torch.nn as nn
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from PIL import Image
from tqdm import tqdm as tqdm
import pandas as pd
import os

In [ ]:
class ResNet50Regresion:
    def __init__(self, batch_size=32, learning_rate=1e-4, epochs=10, device=None):
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.device = device or ('cuda' if torch.cuda.is_available() else 'cpu')

        self.modelo = self.get_model().to(self.device)

        self.criterion = nn.MSELoss()  # Para regresión, usamos error cuadrático medio
        self.optimizer = optim.Adam(self.modelo.parameters(), lr=self.learning_rate)

    def get_model(self):
        modelo_resnet = models.resnet50(pretrained=True)
        for param in modelo_resnet.parameters():
            param.requires_grad = False  # Congelar todas las capas
            
        num_ftrs = modelo_resnet.fc.in_features
        modelo_resnet.fc = nn.Linear(num_ftrs, 1)  # Cambiar la capa final para regresión
        return modelo_resnet

    def fit(self, X_train, y_train):
        self.modelo.train()

        for epoch in range(self.epochs):
            running_loss = 0.0
            for i in range(0, len(X_train), self.batch_size):
                batch_inputs = X_train[i:i+self.batch_size]
                batch_labels = y_train[i:i+self.batch_size]
                inputs = torch.stack(batch_inputs).to(self.device)
                labels = torch.tensor(batch_labels, dtype=torch.float32).to(self.device)
                self.optimizer.zero_grad()
                outputs = self.modelo(inputs)

                loss = self.criterion(outputs.squeeze(), labels)  # Squeeze para eliminar la dimensión extra
                loss.backward()
                self.optimizer.step()

                running_loss += loss.item()


            epoch_loss = running_loss / len(X_train)
            print(f"Epoch {epoch+1}, Loss: {epoch_loss:.4f}")

    def save_model(self, ruta):
        torch.save(self.modelo.state_dict(), ruta)
        print(f"Modelo guardado en {ruta}")

    def predict(self, X_test):
        self.modelo.eval()  # Poner el modelo en modo de evaluación
        predicciones = []

        for img_tensor in X_test:
            img_tensor = img_tensor.unsqueeze(0).to(self.device)  # Agregar una dimensión para batch

            with torch.no_grad():
                output = self.modelo(img_tensor)
                predicciones.append(output.item())

        return predicciones



In [ ]:
transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

# Cargar los datos
lista_paths_train = pd.read_csv("../X_test_paths.csv").iloc[:, 0].tolist()
etiquetas_train = pd.read_csv("../y_test.csv").iloc[:, 0].tolist() 

 # Cargar las imágenes y aplicar las transformaciones
X_train = []
for path in lista_paths_train:
    path = f"../imgs/{path}"
    img = Image.open(path).convert('RGB')
    img_tensor = transform(img)  # Aplicar las transformaciones
    X_train.append(img_tensor)


: 

In [ ]:
modelo = ResNet50Regresion(batch_size=3000, epochs=2, learning_rate=1e-4)

    # Entrenar el modelo
modelo.fit(X_train, etiquetas_train)  # Pasar las imágenes procesadas y las etiquetas

# Guardar el modelo
modelo.save_model("../modelo_resnet50.pth")

voy a entrenar


/Users/dan/miniconda3/envs/ML/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/dan/miniconda3/envs/ML/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


pasito
hola tensor(55.1939, grad_fn=<MseLossBackward0>)
pasito
